# Resolución ejercicio 3.10.29
## Análisis y diseño de algoritmos distribuidos en redes
### Andrés Montoro 5.169.779-1

Show the step-by-step execution of Stages and of UniStages in the ring of Figure 3.3. Indicate for each step, the values know at the candidates.

Se asumen las restricciones R + ID:
- Links bidireccionales
- Conectividad
- Total reliability
- Identificadores

La figura 3.3 es un anillo de n nodos. Por lo tanto vamos a asumir también la siguiente restriccion:
- Anillo

## Stages

### Implementacion

In [ ]:
from pydistsim.algorithm.node_algorithm import NodeAlgorithm, StatusValues
from pydistsim.algorithm.node_wrapper import NodeAccess
from pydistsim.message import Message
from pydistsim.restrictions.communication import BidirectionalLinks
from pydistsim.restrictions.reliability import TotalReliability
from pydistsim.restrictions.communication import MessageOrdering
from pydistsim.restrictions.topological import Connectivity, UniqueInitiator
import random

Q = "I'm your father"


class Stages(NodeAlgorithm):
    default_params = {
        "req_msj" : Q,
    }

    class Status(StatusValues):
        INITIATOR = "INITIATOR"
        IDLE = "IDLE"
        ACTIVE = "ACTIVE"
        DONE = "DONE"

    S_init = (Status.INITIATOR, Status.IDLE, Status.ACTIVE, Status.DONE)
    S_term = (Status.DONE)


    algorithm_restrictions = (
    )



    def initializer(self):
        for node in self.network.nodes():
            node.status = self.Status.IDLE
            node.memory["value"] = random.randint(1, 2*self.network.size())
            node.memory["rank"] = 1
            node.memory["waiting_values"] = []
            node.memory["Q"] = self.default_params["req_msj"]
            node.memory["Yes"] = self.default_params["yes_msj"]
            node.memory["No"] = self.default_params["no_msj"]
        ini_node = self.network.nodes_sorted()[0]
        ini_node.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=ini_node))
        ini_node.status = self.Status.INITIATOR


    @Status.INITIATOR
    def spontaneously(self, node: NodeAccess, message: Message):
        self.send(
            node,
            data=node.memory["Q"],
            destination=list(node.neighbors()), 
            header= Q,
        )
        node.status = self.Status.ACTIVE


    @Status.IDLE
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == Q:
            node.memory["counter"] = 1
            node.memory["tree"].append(message.source)
            self.send( 
                node,
                data= node.memory["Yes"],
                destination=[message.source],
                header= YES,
            )
            if len(list(node.neighbors())) == 1:
                self.send( 
                    node,
                    data=node.memory["value"],
                    destination=node.memory["tree"],
                    header= "value",
                )
                node.status = self.Status.DONE
            else:
                self.send( 
                    node,
                    data= node.memory["Q"],
                    destination=list(set(node.neighbors()) - {message.source}),
                    header= Q,
                )
                node.status = self.Status.ACTIVE
        else:
            msj = 'Unexpected message in idle-receiving ' + message.header + " from " + str(message.source) + " , content: " + str(message.data)
            raise Exception(msj)


    @Status.ACTIVE
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == Q:
            self.send(
                node,
                data= node.memory["No"],
                destination=[message.source],
                header= Q,
            )
        else:
            msj = 'Unexpected message in active-receiving ' + message.header + " from " + str(message.source) + " , content: " + str(message.data)
            raise Exception(msj)
    
    @Status.DONE
    def default(self, *args, **kwargs):    
        pass

### Ejecución


In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from pydistsim import NetworkGenerator, Simulation
from pydistsim.logging import set_log_level, LogLevels, enable_logger
from pydistsim.network.behavior import NetworkBehaviorModel
from pydistsim.gui import drawing as draw


net_gen = NetworkGenerator(100, directed=False)
net = net_gen.generate_random_network()
sim = Simulation(net, check_restrictions=True)
sim.algorithms = (Stages,)
set_log_level(LogLevels.INFO)
enable_logger()

behaviour = NetworkBehaviorModel(message_ordering=True, message_loss_indicator=None, )
sim.network.behavioral_properties = behaviour

fig = draw.draw_current_state(sim)
fig

sim.run()
#fig = draw.draw_current_state(sim)
#fig

values = []
for node in net.nodes():
    values.append((node.memory["rank"], node.memory["value"]))

piso = -1
for (rank, val) in sorted(values):
    if piso > val:
        assert False, "Error en el ranking: " + str(val)
    piso = val



## UniStages


### Implementacion

### Ejecucion

## Otros

In [ ]:
sim.reset()
plt.close()